# Robust Identification of Investor Beliefs

by [Xiaohong Chen](https://economics.yale.edu/people/faculty/xiaohong-chen), [Lars Peter Hansen](http://larspeterhansen.org/) and [Peter G. Hansen](https://mitsloan.mit.edu/phd/students/peter-hansen).

The latest draft of the paper can be found [here](http://larspeterhansen.org/research/papers/).

Notebook by: Han Xu

## 1. Overview

This notebook provides source code and explanations for how we solve the dynamic problem in Section 5. It also provides source code for the tables in Section 6. 

## 2. Intertemporal Divergence

Recall Problem 5.4 in the paper:

\begin{equation}
\mu = \min_{N_1\in \mathcal{N}} \mathbb{E}\left(N_1\left[g(X_1)+\xi\log N_1 + v_1\right]\mid \mathfrak{J}_0\right) - v_0
\end{equation}
subject to constraints:
\begin{equation}
\mathbb{E}\left[N_1 f(X_1)\mid\mathfrak{J}_0\right] = 0
\end{equation}

By Proposition 5.6, this problem can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_{\lambda_0}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\lambda_0\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{J}_0\right)
\end{equation}

where
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

The implied solution for the probablity distortion is:

\begin{equation}
N_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\lambda^*_0(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

where $\lambda^*_0$ is the optimizing choice for $\lambda_0$ and $\left(\epsilon^*,e_0^*\right)$ are selected to that the resulting $\sf Q$ implies that the process is stochastically stable. The conditional expectation implied by the bound is

\begin{equation}
\mathbb{E}\left[N_1^*g(X_1)\mid \mathfrak{J}_0\right]
\end{equation}

which in turn implies a bound on the unconditional expectation equal to

\begin{equation}
\int \mathbb{E}\left[N_1^*g(X_1)\mid\mathfrak{J}_0\right]d \sf Q_0^*
\end{equation}

The implied relative entropy is

\begin{equation}
\int \mathbb{E}\left(N_1^*\log N_1^*\mid \mathfrak{J}_0\right)d \sf Q_0^*
\end{equation}

### 2.1. Code Implementation

In [ ]:
# Load packages
import time
from utilities_s8 import *
from plotting_module import *

#### 2.1.1 Data
The file “UnitaryData.csv” contains the following variables:

- The first four columns contain Euler equation errors from the unitary risk aversion model corresponding to the 3-month T-bill rate, the market excess return, the SMB excess return, and the HML excess return respectively. Under a feasible belief distortion, all four of these variables should have expectation zero (conditional or unconditional).


- The column “d.p” contains the dividend-price ratio for the CRSP value weighted index, computed at the start of the return period. Hence functions of d.p[i] (i.e. quantile indicator functions) are valid instruments for the returns in row i.


- The final column “log.RW” contains values of the logarithmic return on wealth. This is the random variable who’s expectation we are intersted in bounding.

All returns are quarterly and inflation adjusted.

In [11]:
# Load data
data = pd.read_csv('UnitaryData.csv')
# Show statistics of the data
data.describe()

,Rf,Rm-Rf,SMB,HML,d.p,log.RW
count,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000
mean,-0.010974,0.010974,0.004992,0.009498,0.029250,0.018864
std,0.086116,0.086116,0.052512,0.058218,0.010055,0.084452
min,-0.186245,-0.356102,-0.139624,-0.269700,0.010571,-0.312496
25%,-0.063218,-0.029274,-0.026004,-0.024469,0.020545,-0.021638
50%,-0.028582,0.028582,0.000834,0.008563,0.029106,0.031677
75%,0.029274,0.063218,0.035165,0.036895,0.036495,0.069903
max,0.356102,0.186245,0.169266,0.234932,0.058402,0.210429


#### 2.1.2 Construcct f and g

#### 2.1.3 Computational Strategy

In our application, we just have 3 distinct states, so we can represent the function $e(\cdot)$ as a 3-dimensional vector. Additionally, we are free to impose the normalization $e_1=1$. We can solve the dual problem numerically by something analogous to value function iteration for $e=(1,e_2,e_3)$. Here is the iteration:

1\. Guess $e=(1,1,1)$.

2\. For $k \in \{1,2,3\}$, solve
\begin{equation}
v_k = \min_{\hat{\lambda}} \hat{\mathbb{E}}\left(\exp \left[-\frac{1}{\xi}g(X) + \hat{\lambda}f(X)\right]e(X)\mid \text{state } k\right)
\end{equation}

3\. Store
\begin{align*}
\hat{e} &= v/v_1 \\
\hat{\epsilon} &= v_1 \\
\text{error} &= \|\hat{e}-e\|
\end{align*}

4\. Set $e = \hat{e}$.

5\. Iterate steps 2-4 until error is arbitrarily close to zero.

Once we have (approximately) stationary values for $\epsilon$ and $e$, we can form the conditional belief distortion
\begin{equation}
N_1 = \frac{1}{\epsilon} \exp \left[-\frac{1}{\xi}g(X)+\lambda_0 \cdot f(X)\right]\frac{e_1}{e_0}
\end{equation}

To obtain the unconditional relative entropy, we need to average across states using the implied stationary distribution coming from the distorted probabilities. Define a $3\times 3$ matrix $\tilde{P}$ by 
$$
\tilde{P}_{i,j} = \hat{\mathbb{E}}\left[N_1 \mathcal{1}\left(\text{state tomorrow = j}\right)\mid \text{state tomorrow = j}\right]
$$

We should have that $\tilde{P}$ is a transition probability matrix, so $\tilde{P}\mathcal{1}=\mathcal{1}$. Next, solve for the stationary distribution $\pi\in \mathbb{R}^3$ as the dominant left eigenvector of $\tilde{P}$, i.e.
\begin{equation}
\pi^\prime \tilde{P} = \pi^\prime
\end{equation}

Then, the unconditional relative entropy can be computed as
\begin{equation}
\text{RE}(\xi) = \sum_{k=1}^{3}\hat{\mathbb{E}}\left[N_1\log N_1 \mid \text{state k}\right]\cdot \pi_k
\end{equation}

#### 2.1.4 Code Implementation

In [2]:
# Set model paramters

ξ = 0.2  # 10% higher than the minimum RE
tol = 1e-10

# ξ = 100.  # Minimal RE
# tol = 1e-12

solver_s8 = InterDivConstraint(tol,max_iter=1000)

time_start = time.time() 
result = solver_s8.iterate(ξ,lower=True)
result_upper = solver_s8.iterate(ξ,lower=False)

# Print iteration information
print("--- Iteration Ends ---")
print("ξ = %s" % ξ)
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

# Print converged parameter results
print("\n")
print("--- Converged vlues for the lower bound problem ---")
print("ϵ: %s" % result['ϵ'])
print("e: %s" % result['e'])
print("λ: %s" % result['λ'])
print("μ: %s" % result['μ'])

print(" ")
print("--- Converged vlues for the upper bound problem ---")
print("ϵ: %s" % result_upper['ϵ'])
print("e: %s" % result_upper['e'])
print("λ: %s" % result_upper['λ'])
print("μ: %s" % result_upper['μ'])

# Print transition probability matrix under the original empirical probability
print("\n")
print("--- Transition Probability Matrix (Original) ---")
print(result['P'])

# Print transition probability matrix under distorted probability, lower bound
print(" ")
print("--- Transition Probability Matrix (Distorted, lower bound problem) ---")
print(result['P_tilde'])

# Print transition probability matrix under distorted probability, upper bound
print(" ")
print("--- Transition Probability Matrix (Distorted, upper bound problem) ---")
print(result_upper['P_tilde'])


# Print stationary distribution under the original empirical probability
print("\n")
print("--- Stationary Distribution (Original) ---")
print(result['π'])

# Print stationary distribution under distorted probability, lower bound
print(" ")
print("--- Stationary Distribution (Distorted, lower bound problem) ---")
print(result['π_tilde'])

# Print stationary distribution under distorted probability, upper bound
print(" ")
print("--- Stationary Distribution (Distorted, upper bound problem) ---")
print(result_upper['π_tilde'])

# Print relative entropy
print("\n")
print("--- Relative Entropy (lower bound problem) ---")
print("E[MlogM|state 1] = %s " % result['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result['RE_cond'][2])
print("E[MlogM]         = %s " % result['RE'])

# Print relative entropy
print(" ")
print("--- Relative Entropy (Upper bound problem) ---")
print("E[MlogM|state 1] = %s " % result_upper['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result_upper['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result_upper['RE_cond'][2])
print("E[MlogM]         = %s " % result_upper['RE'])

# Print conditional moment & bounds
print("\n")
print("--- Conditional Moment (Original) ---")
print("E[g(X)|state 1]  = %s " % (result['moment_cond'][0]))
print("E[g(X)|state 2]  = %s " % (result['moment_cond'][1]))
print("E[g(X)|state 3]  = %s " % (result['moment_cond'][2]))
print(" ")
print("--- Conditional Moment (Lower bound) ---")
print("E[Mg(X)|state 1] = %s " % (result['moment_bound_cond'][0]))
print("E[Mg(X)|state 2] = %s " % (result['moment_bound_cond'][1]))
print("E[Mg(X)|state 3] = %s " % (result['moment_bound_cond'][2]))
print(" ")
print("--- Conditional Moment (Upper bound) ---")
print("E[Mg(X)|state 1] = %s " % (result_upper['moment_bound_cond'][0]))
print("E[Mg(X)|state 2] = %s " % (result_upper['moment_bound_cond'][1]))
print("E[Mg(X)|state 3] = %s " % (result_upper['moment_bound_cond'][2]))

# Print unconditional moment & bounds
print("\n")
print("--- Unconditional Moment (Original) ---")
print("E[g(X)]  = %s " % (result['moment']))
print(" ")
print("--- Unconditional Moment (Lower bound) ---")
print("E[Mg(X)] = %s " % (result['moment_bound']))
print(" ")
print("--- Unconditional Moment (Upper bound) ---")
print("E[Mg(X)] = %s " % (result_upper['moment_bound']))

--- Iteration Ends ---
ξ = 0.2
Time spent: 2.9081 seconds ---


--- Converged vlues for the lower bound problem ---
ϵ: 0.9461095168590227
e: [1.         0.3943727  0.14879927]
λ: [-0.704944    2.704944   -0.35540586 -0.84585694 -0.41751483  2.41751483
  0.70692734 -6.33311769  0.7196623   1.2803377  -3.66748022 -7.32226842]
μ: 0.01107938965619576
 
--- Converged vlues for the upper bound problem ---
ϵ: 1.0047686045117912
e: [1.         0.50569989 0.27007686]
λ: [ 4.02073828 -2.02073828 -0.31787233 -0.52909156  4.18684359 -2.18684359
  0.61436948 -5.53936191  5.63627713 -3.63627713 -3.26453449 -8.5249151 ]
μ: -0.0009514541467768203


--- Transition Probability Matrix (Original) ---
[[0.96341463 0.03658537 0.        ]
 [0.04878049 0.87804878 0.07317073]
 [0.         0.08433735 0.91566265]]
 
--- Transition Probability Matrix (Distorted, lower bound problem) ---
[[0.98335598 0.01664402 0.        ]
 [0.09351864 0.87567959 0.03080177]
 [0.         0.18772092 0.81227908]]
 
--- Transition Pr

#### 2.1.5 Plots

## 3. Tables in Section 6